TODO: ADD Description

In [28]:
import math
import datetime
import random

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


random.random()

0.2445444198310759

In [128]:

def getEuclideanDistance(v1: list, v2: list) -> float:
	distance = 0.0
	for i in range(len(v1)):
		distance += (v1[i] - v2[i])**2
	return math.sqrt(distance)

def getNumbersSummingToOne(count: int) -> list:
    r = [random.random() for i in range(0, count)]
    randomSum = sum(r)
    return [ i / randomSum for i in r ]



Import DS

In [111]:
PATH_FILE_DS = './file/ds-fcm.csv'
df = pd.read_csv(PATH_FILE_DS, header=None, names=['x', 'y'])

'''
    TODO: 2021-11-27 - Use the whole DF
'''
df = df.loc[0:30]


In [114]:
def getCentroid(X: np.array, uColumn: np.array, m: int) -> np.array:
    '''
        Calculates coordinates of one of the k centroids.

        - X: Inputs data matrix;
        - uColumn: Weights Column: 01 column of the partition Matrix that holds the current weights of the k-th class for each i-th point;
        - m: Centroid calculation parameter;
    '''

    poweredU = np.power(uColumn, m)
    weightedX = poweredU * X
    dimension = X.shape[1]
    centroid = np.zeros(dimension)
    for i in range(dimension):
        centroid[i] = weightedX[:, i].sum() / poweredU
    return centroid


def getCentroids(X: np.array, U: np.array, m: int, k: int) -> np.array:
    '''
        Calculates and return the coordinates of all k centroids.
    '''

    dimension = X.shape[1]
    centroids = np.zeros(k, dimension)
    for i in range(k):
        uColumn = U[:, i]
        centroids[i, :] = getCentroid(X=X, uColumn=uColumn, m=m)
    
    return centroids

def getPartitionMatrix(X: np.array, centroids: np.array, m: int) -> n.array:
    '''
        TODO: 2021-11-29 - ADD Description
    '''
    
    # Build matrix of squared euclidean distances (of each row related to each centroid)
    getRowSquaredDistances = lambda x: [getEuclideanDistance(x, c)**2 for c in centroids]
    squaredDistances = np.apply_along_axis(lambda x: getRowSquaredDistances(x), 1, X)

    # Build U: Partition matrix
    squaredDistSum = squaredDistances.sum(axis=1)
    getMembershipDegree = lambda dist, allDistancesSum: 1 / (dist / allDistancesSum)**(2 / (m - 1))
    getPartitionRow = lambda row: [getMembershipDegree(dist, squaredDistSum[i]) for i, dist in enumerate(row)]
    return np.apply_along_axis(lambda x: getPartitionRow(x), 1, squaredDistances)


k = 4
m = 2
n = df.shape[0]
maxEpochs = 1000

# Gerate random initial U
U = np.zeros( (n, k) )
U = np.apply_along_axis(lambda _: getNumbersSummingToOne(k), 1, U)

haveChanged = True
epoch = 0

while haveChanged and epoch < maxEpochs:
    epoch = epoch + 1

    # Calculate centroids
    centroids = getCentroids(X=df.values, U=U, m=m, k=k)

    # Update clustering
    newU = getPartitionMatrix(X=df.values, centroids=centroids, m=m)
    diff = U != newU
    U = newU

    haveChanged = np.sum(diff, dtype=int) > 0


array([0., 0.])

In [157]:
centroids = np.array([ (12, 4.5453), (8, 23.1) ])
# X = df.values[:2, :]
X = df.values
cIndex = 1
# print(X)
# centroids
for i in range(4):
    print(2)



2
2
2
2


In [154]:
# foo = [(1, 'a'), (1, 'b'), (1, 'c')]
foo = [1, 23, 34, 4]

# [print(i, j) for i, j in foo]

for i, j in e(foo):
    print(i, j)

TypeError: cannot unpack non-iterable int object